In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Some countries and most states are branched in CONVENIENT datas

In [ ]:
import warnings; warnings.simplefilter('ignore')

> Exsample of global data

In [ ]:
path = '../input/covid19-data-from-john-hopkins-university/'
pd.read_csv(path + 'CONVENIENT_global_confirmed_cases.csv').iloc[0:5,9:17]

> Exsample of US data

In [ ]:
pd.read_csv(path + 'CONVENIENT_us_confirmed_cases.csv').iloc[:5,:7]

## So, I tried to use meta-data to integrate branches of countries or states

> meta-data 

In [ ]:
pd.read_csv(path + 'CONVENIENT_global_metadata.csv').iloc[7:15]

### For global CONVENIENT data

In [ ]:
# Load data except for sub-header
global_confirmed = pd.read_csv(path + 'CONVENIENT_global_confirmed_cases.csv'
                               ,skiprows=[1])
global_deaths = pd.read_csv(path + 'CONVENIENT_global_deaths.csv', skiprows=[1])

# Check branched countries and num_branches of them
global_meta = pd.read_csv(path + 'CONVENIENT_global_metadata.csv')

# Pick up countries with branches
branched = list(set(
    global_meta[global_meta['Country/Region'].duplicated()]['Country/Region']))
num_countries = global_meta['Country/Region'].value_counts()

# Sum up branches and delete
for c in branched:
    num_branch = num_countries[c]
    prefix = c + '.'

    for i in range(1,num_branch):
        col_name = prefix + str(i)
        global_confirmed[c] += global_confirmed[col_name]
        global_deaths[c] += global_deaths[col_name]

    global_confirmed.drop([prefix+str(i) for i in range(1,num_branch)],
                          axis=1,inplace=True)
    global_deaths.drop([prefix+str(i) for i in range(1,num_branch)],
                       axis=1,inplace=True)

# Convert 'Country/Region' to datetime format and set to index
global_confirmed['Country/Region'] = pd.to_datetime(
    global_confirmed['Country/Region'])
global_confirmed = global_confirmed.rename(columns={'Country/Region':'date'}
                                          ).set_index('date')
global_deaths['Country/Region'] = pd.to_datetime(global_deaths['Country/Region'])
global_deaths = global_deaths.rename(columns={'Country/Region':'date'}
                                    ).set_index('date')

#### Let's chek outcome

In [ ]:
global_confirmed.iloc[:5,6:12]

### For US COVENIENT datas - same as global datas

In [ ]:
# Load data except for sub-header
us_confirmed = pd.read_csv(path + 'CONVENIENT_us_confirmed_cases.csv',
                           skiprows=[1])
us_deaths = pd.read_csv(path + 'CONVENIENT_us_deaths.csv',skiprows=[1])

# Check num_branches of each Province_State
us_meta = pd.read_csv(path + 'CONVENIENT_us_metadata.csv')

branched_states = list(set(
    us_meta[us_meta['Province_State'].duplicated()]['Province_State']))
num_admins = us_meta['Province_State'].value_counts()

# Sum up branches and delete
for s in branched_states:
    num_branch = num_admins[s]
    prefix = s + '.'

    for i in range(1,num_branch):
        col_name = prefix + str(i)
        us_confirmed[s] += us_confirmed[col_name]
        us_deaths[s] += us_deaths[col_name]

    us_confirmed.drop([prefix+str(i) for i in range(1,num_branch)],
                      axis=1,inplace=True)
    us_deaths.drop([prefix+str(i) for i in range(1,num_branch)],
                   axis=1,inplace=True)

# Convert 'Province_State' column to datetime format and set to index
us_confirmed['Province_State'] = pd.to_datetime(us_confirmed['Province_State'])
us_confirmed = us_confirmed.rename(columns={'Province_State':'date'}
                                  ).set_index('date')
us_deaths['Province_State'] = pd.to_datetime(us_deaths['Province_State'])
us_deaths = us_deaths.rename(columns={'Province_State':'date'}).set_index('date')

#### Let's check outcome

In [ ]:
us_confirmed.iloc[:5,:7]